In [ ]:
##import Pkg; Pkg.add("SpeedyWeather")
using Pkg
#Pkg.status("SpeedyWeather")
Pkg.add(url="https://github.com/SpeedyWeather/SpeedyWeather.jl",rev="main")
#Pkg.status("SpeedyWeather")

    Updating git-repo `https://github.com/SpeedyWeather/SpeedyWeather.jl`
   Resolving package versions...
   Installed ModelParameters ─ v0.4.2
Precompiling CUDA_Driver_jll...
    754.0 ms  ✓ CUDA_Driver_jll
  1 dependency successfully precompiled in 3 seconds. 25 already precompiled.
    Updating `~/Python_scripts/Project.toml`
  [9e226e20] ~ SpeedyWeather v0.15.0 ⇒ v0.16.0 `https://github.com/SpeedyWeather/SpeedyWeather.jl#main`
    Updating `~/Python_scripts/Manifest.toml`
  [85c772de] + AbstractNumbers v0.2.5
  [b0b7db55] + ComponentArrays v0.15.29
  [b152e2b5] + CompositeTypes v0.1.4
  [5b8099bc] + DomainSets v0.7.16
  [bf96fef3] + FieldMetadata v0.3.1
  [4c728ea3] + Flatten v0.4.3
  [d9f16b24] + Functors v0.5.2
  [299715c1] - MarchingCubes v0.1.11
⌃ [4744a3fa] + ModelParameters v0.4.2
  [efcf1570] + Setfield v1.1.2
  [9e226e20] ~ SpeedyWeather v0.15.0 ⇒ v0.16.0 `https://github.com/SpeedyWeather/SpeedyWeather.jl#main`
  [b8865327] - UnicodePlots v3.7.2
        Info Packages marke

In [1]:
using SpeedyWeather
# components
spectral_grid = SpectralGrid(trunc=21, nlayers=8)
output = NetCDFOutput(spectral_grid, PrimitiveWetModel, path="/Users/vidale/SpeedyWeather", id="Climate_Ocean_Land", output_dt=Minute(180))
add!(output, SpeedyWeather.TemperatureOutput(), SpeedyWeather.LargeScalePrecipitationOutput(), SpeedyWeather.ConvectivePrecipitationOutput())  # output also temperature and precip

# Some parameterisation choices
vertical_advection = WENOVerticalAdvection(spectral_grid)
large_scale_condensation = ImplicitCondensation(spectral_grid)
convection = SimplifiedBettsMiller(spectral_grid)
temperature=LandBucketTemperature(spectral_grid)
soil_moisture=LandBucketMoisture(spectral_grid)
land = LandModel(spectral_grid; temperature, soil_moisture)
ocean = SlabOcean(spectral_grid, mixed_layer_depth=50)
#longwave_radiation = NBandRadiation(spectral_grid, nbands=5)

#Try to fix ocean albedo
#albedo = Albedo(AlbedoClimatology(spectral_grid), AlbedoClimatology(spectral_grid))
albedo = Albedo(GlobalConstantAlbedo(spectral_grid, albedo=0.06), AlbedoClimatology(spectral_grid))
# create model, initialize, run

#time_stepping = Leapfrog(spectral_grid, Δt_at_T31=Minute(20))
time_stepping = Leapfrog(spectral_grid, Δt_at_T31=Minute(40)) #CPL this is the standar time step

### Entirely new run
model = PrimitiveWetModel(spectral_grid; vertical_advection, time_stepping, large_scale_condensation, convection, land, ocean, albedo, output=output)

### Continuation run
#initial_conditions = StartFromFile(path="/Users/vidale/SpeedyWeather", id="Climate_Ocean_Land",run_number=24)   # or specify `path` kwarg if not in this folder
#model = PrimitiveWetModel(spectral_grid; initial_conditions, vertical_advection, time_stepping, large_scale_condensation, convection, land, ocean, albedo, output=output)

add!(model, SpeedyWeather.AllOutputVariables()...)
simulation = initialize!(model)
# now offset via set! with add=true
set!(albedo.land, (λ, φ) -> 0., add=true)
run!(simulation, period=Day(30*365), output=true)

Weather is speedy: run_Climate_Ocean_Land_0001 100% Time: 1:30:40 (476.40 years/day)


Simulation{PrimitiveWetModel}
├ prognostic_variables::PrognosticVariables{...}
├ diagnostic_variables::DiagnosticVariables{...}
└ model::PrimitiveWetModel{...}